In [3]:
import pandas as pd 

In [6]:
df_triage = pd.read_csv('../triage/data/triage_initial_preprocessed.csv', sep=";", decimal=",")
df_harvard = pd.read_csv('../harvard/data/harvard_cleaned.csv', sep=";", decimal=",")

['Advair_ra', 'Albuterol_ra', 'Assmnt', 'Beclomethasone_ra', 'Budesonide_ra', 'Combivent_ra', 'Elig03', 'Flucticasone_ra', 'Formoterol_ra', 'Ipratropium_ra', 'Levaquin_ra', 'Mometasone_ra', 'Oxygen_ra', 'Serevent_ra', 'Singulair_ra', 'Socio07', 'Tiotropium_ra', 'Xopenex_ra', 'bclinpt03', 'bclinpt04', 'bclinpt05', 'bclinpt06', 'bclinpt07', 'bclinpt08', 'bclinpt09', 'bclinpt10', 'bclinpt11', 'bclinpt12', 'bclinpt12a', 'bclinpt13', 'bclinpt14', 'bclinpt15', 'bclinpt16', 'bclinpt17', 'bclinpt18', 'bclinpt19', 'bclinpt21', 'bclinpt22', 'bclinpt23', 'bclinpt24', 'bclinpt26', 'bclinpt28', 'bclinpt30', 'bclinpt32', 'bclinpt34', 'bclinpt36', 'bclinpt38', 'bclinpt40', 'bclinra01', 'bclinra02', 'bclinra03', 'bclinra07', 'bclinra08', 'bclinra09', 'bclinra10', 'bclinra101', 'bclinra102', 'bclinra103', 'bclinra104', 'bclinra11', 'bclinra14', 'bclinra16', 'bclinra202', 'bclinra202b_2', 'bclinra202b_5', 'bclinra202c_2', 'bclinra202c_5', 'bclinra203', 'bclinra203b_2', 'bclinra203b_5', 'bclinra203c_2', 

# Select Relevant Features

In [7]:
# TODO include medications and help with daily activities

relevant_vars_triage = [
    '2+ Exa. last year', # bclinpt22
    'Age', # dem02
    'Asthma', # bclinpt14
    'BMI', # bmi - from height (bclinra01)  and weight (bclinra02)
    'Congestive Heart Failure', # bclinpt07
    'Coronary Artery Disease', # blcinpt06
    'Cough details', # bclinpt30
    'Current Temperature', # could turn this into fever (bclinpt34)
    'Diabetes', # bclinpt15
    'Exacerbation', # fclinra08
    # 'FEV1', # bclinra202
    '% FEV1 a', # bclinra202c
    '% FEV1 b', # bclinra202c_2
    'Gender, Male', # dem03
    'GOLD', # bclinra11
    'Height', # bclinra01
    'Help Daily Activities', # An14
    'Hospitalized last year', # bclinpt24
    'Hypertension', # bclinpt04
    'ICU last year', # bclinpt26
    'Kidney disease', # bclinpt11
    'Lives Alone', # socio06
    'MMRC a', # mmrc
    'MMRC b', # mmrc
    'Night Symptoms', # psqidistb (sleep disturbance score)
    'Oxygen Therapy', # Oxygen_ra
    'Pill to control Exa.',
    'Daily Inhaler',
    'Rescue Inhaler',
    'Dyspnea details', #bclinpt28
    'Sputum details', # bclinpt32
    'Weight', # bclinra02
    'Wheezing details', # bclinpt38
    'Smoker', # bclinpt34
]

df_triage_selected = df_triage[relevant_vars_triage]

relevant_vars_harvard = [
    'dem08_8',
    'bclinpt22',
    'dem02',
    'bclinpt14',
    'bclinra01',
    'bclinra02',
    'bclinpt07',
    'bclinpt06',
    'bclinpt30',
    'bclinpt15',
    'fclinra08',
    'bclinra202',
    'bclinra202c_2',
    'bclinra202c_5',
    'dem03',
    'bclinra11',
    'bclinpt24',
    'bclinpt04',
    'bclinpt26',
    'bclinpt11',
    'mmrc',
    'Oxygen_ra',
    'psqidistb',
    'bclinpt28',
    'bclinpt32',
    'bclinpt38',
    'Advair_ra',
    'Albuterol_ra',
    'Beclomethasone_ra',
    'Budesonide_ra',
    'Combivent_ra',
    'Flucticasone_ra',
    'Formoterol_ra',
    'Ipratropium_ra',
    'Levaquin_ra',
    'Mometasone_ra',
    'Serevent_ra',
    'Singulair_ra',
    'Tiotropium_ra',
    'Xopenex_ra',
    'socio06',
    'bclinpt34',
]

df_harvard_selected = df_harvard[relevant_vars_harvard]

In [8]:
df_triage_simple_harmo = df_triage_selected.copy()
df_harvard_simple_harmo = df_harvard_selected.copy()

## TRIAGE
# fever == 1 if (temp > 38°C)
df_triage_simple_harmo['Fever'] = 0
df_triage_simple_harmo.loc[df_triage_simple_harmo['Current Temperature'] > 38, 'Fever'] = 1
df_triage_simple_harmo.drop(columns=['Current Temperature'], inplace=True)


## HARVARD

# TODO there is still a difference in FEV1 scale
# TODO % FEV1 b and % FEV1 and how they are created

# BMI using bclinra01 and bclinra02
df_harvard_simple_harmo['BMI'] = df_harvard_simple_harmo['bclinra02'] / (df_harvard_simple_harmo['bclinra01'] / 100) ** 2

# turn bclinpt22 into 2+ exa last year (2+ exa last year = bclinpt22 >= 3)
df_harvard_simple_harmo['2+ Exa. last year'] = 0
df_harvard_simple_harmo.loc[df_harvard_simple_harmo['bclinpt22'] >= 3, '2+ Exa. last year'] = 1
df_harvard_simple_harmo.drop(columns=['bclinpt22'], inplace=True)

# hospitalized last year
df_harvard_simple_harmo['Hospitalized last year'] = 0
df_harvard_simple_harmo.loc[df_harvard_simple_harmo['bclinpt24'] >= 2, 'Hospitalized last year'] = 1
df_harvard_simple_harmo.drop(columns=['bclinpt24'], inplace=True)
# icu last year
df_harvard_simple_harmo['ICU last year'] = 0
df_harvard_simple_harmo.loc[df_harvard_simple_harmo['bclinpt26'] >= 2, 'ICU last year'] = 1
df_harvard_simple_harmo.drop(columns=['bclinpt26'], inplace=True)

# not using smoker variables as everyone was a smoker in harvard

# lives alone (socio06 is Never Married (1),Separated (4), Divorced (5) or Widowed(6))
df_harvard_simple_harmo['Lives Alone'] = 0
df_harvard_simple_harmo.loc[df_harvard_simple_harmo['socio06'].isin([1, 4, 5, 6]), 'Lives Alone'] = 1
df_harvard_simple_harmo.drop(columns=['socio06'], inplace=True)

# Medications

In [9]:
df_harvard_med_harmo = df_harvard_simple_harmo.copy()

# provided medications are categorized according to what they belong to (pill, rescue inhaler, controller inhaler)
pills = ['Levaquin_ra', 'Singulair_ra']
rescue_inhalers = ['Albuterol_ra', 'Xopenex_ra', 'Ipratropium_ra', 'Combivent_ra']
daily_inhalers = ['Advair_ra', 'Serevent_ra', 'Formoterol_ra', 'Budesonide_ra', 'Flucticasone_ra', 'Beclomethasone_ra', 'Mometasone_ra', 'Tiotropium_ra']

df_harvard_med_harmo['Pill to control Exa.'] = 0
df_harvard_med_harmo['Rescue Inhaler'] = 0
df_harvard_med_harmo['Daily Inhaler'] = 0

for pill in pills:
    df_harvard_med_harmo.loc[df_harvard_med_harmo[pill] == 1, 'Pill to control Exa.'] = 1
for rescue_inhaler in rescue_inhalers:
    df_harvard_med_harmo.loc[df_harvard_med_harmo[rescue_inhaler] == 1, 'Rescue Inhaler'] = 1
for daily_inhaler in daily_inhalers:
    df_harvard_med_harmo.loc[df_harvard_med_harmo[daily_inhaler] == 1, 'Daily Inhaler'] = 1



# drop the original medication columns
df_harvard_med_harmo.drop(columns = ['Levaquin_ra', 'Singulair_ra', 'Albuterol_ra', 'Xopenex_ra', 'Ipratropium_ra', 'Combivent_ra', 'Advair_ra', 'Serevent_ra', 'Formoterol_ra', 'Budesonide_ra', 'Flucticasone_ra', 'Beclomethasone_ra', 'Mometasone_ra', 'Tiotropium_ra'], inplace = True)

# Renaming

In [10]:
harvard_rename_dict = {
    'dem08_8': 'On disability',
    'dem02': 'Age',
    'bclinpt14': 'Asthma',
    'bclinra01': 'Height',
    'bclinra02': 'Weight',
    'bclinpt07': 'Congestive Heart Failure',
    'bclinpt06': 'Coronary Artery Disease',
    'bclinpt30': 'Worse cough',
    'bclinpt15': 'Diabetes',
    'fclinra08': 'Exacerbation',
    'bclinra202': 'FEV1',
    'bclinra202c_2': '% FEV1',
    'bclinra202c_5': '% FEV1 b',
    'dem03': 'Gender, Male',
    'bclinra11': 'GOLD',
    'bclinpt04': 'Hypertension',
    'bclinpt11': 'Kidney disease',
    'mmrc': 'MMRC',
    'Oxygen_ra': 'Oxygen Therapy',
    'psqidistb': 'PSQI Disturbance',
    'bclinpt28': 'Worse dyspnea',
    'bclinpt32': 'Increase in sputum',
    'bclinpt38': 'Wheezing',
    'bclinpt34': 'Fever',
    '% FEV1 a': '% FEV1',
    "Advair_ra": "Advair",
    "Albuterol_ra": "Albuterol",
    "Beclomethasone_ra": "Beclomethasone",
    "Budesonide_ra": "Budesonide",
    "Combivent_ra": "Combivent",
    "Elig02": "Exa. last 2 months",
    "Elig03": "No treatment last 3 days",
    "Flucticasone_ra": "Flucticasone",
    "Formoterol_ra": "Formoterol",
    "Ipratropium_ra": "Ipratropium",
    "Levaquin_ra": "Levaquin",
    "Mometasone_ra": "Mometasone",
    "Oxygen_ra": "Oxygen Therapy",
    "Serevent_ra": "Serevent",
    "Singulair_ra": "Singulair",
    "Socio07": "Education level",
    "Tiotropium_ra": "Tiotropium",
    "Xopenex_ra": "Xopenex",
    "bclinpt03": "Non COPD conditions",
    "bclinpt04": "Hypertension",
    "bclinpt05": "Chest pain",
    "bclinpt06": "Coronary Artery Disease",
    "bclinpt07": "Congestive Heart Failure",
    "bclinpt08": "Myocardial infarction",
    "bclinpt09": "Stroke / TIA",
    "bclinpt10": "Liver disease",
    "bclinpt11": "Kidney disease",
    "bclinpt12": "Arthritis / rheumatism",
    "bclinpt12a": "Osteoarthritis",
    "bclinpt13": "Migraines",
    "bclinpt14": "Asthma",
    "bclinpt15": "Diabetes",
    "bclinpt16": "Depression",
    "bclinpt17": "Anxiety",
    "bclinpt18": "Sleep disorder",
    "bclinpt19": "Cancer",
    "bclinpt21": "Length COPD",
    "bclinpt22": "Exa. last 12 months",
    "bclinpt23": "Exa. last 2 months",
    "bclinpt24": "Hosp. last 12 months",
    "bclinpt26": "ER visits last 12 months",
    "bclinpt28": "Worse dyspnea",
    "bclinpt30": "Worse cough",
    "bclinpt32": "Increase in sputum",
    "bclinpt34": "Fever",
    "bclinpt36": "Breathing faster",
    "bclinpt38": "Wheezing",
    "bclinpt40": "Other symptoms",
    "bclinra01": "Height",
    "bclinra02": "Weight",
    "bclinra03": "COPD type",
    "bclinra07": "COPD state",
    "bclinra101": "FEV1",
    "bclinra102": "FVC",
    "bclinra103": "PEF",
    "bclinra104": "FEV1/FVC x 100",
    "bclinra11": "GOLD",
    "bclinra14": "6MWT",
    "bclinra16": "Enrolled as Exacerbator",
    "bclinra202": "FEV1 b",
    "bclinra202b_2": "Pred. FEV1 a",
    "bclinra202b_5": "Pred. FEV1 b",
    "bclinra202c_2": "% FEV1 a",
    "bclinra202c_5": "% FEV1 b",
    "bclinra203": "FVC b",
    "bclinra203b_2": "Pred. FVC a",
    "bclinra203b_5": "Pred. FVC b",
    "bclinra203c_2": "% FVC a",
    "bclinra203c_5": "% FVC b",
    "bclinra204": "PEF b",
    "bclinra204b_2": "Pred. PEF a",
    "bclinra204b_5": "Pred. PEF b",
    "bclinra204c_2": "% PEF a",
    "bclinra204c_5": "% PEF b",
    "bclinra205": "FEV1/FVC x 100 a",
    "bclinra205b_2": "Pred. FEV1/FVC x 100 a",
    "bclinra205b_5": "Pred. FEV1/FVC x 100 b",
    "dem02": "Age",
    "dem03": "Gender, Male",
    "dem05": "Race",
    "elig09": "Pack years",
    "fclinra08": "Exacerbation",
    "mmrc": "MMRC",
    "socio06": "Relationship status",
    'dem08_1': "Homemaker",
    'dem08_16': "On Leave of absence",
    'dem08_2': "Unemployed",
    'dem08_32': "Full-time employed",
    'dem08_4': "Retired",
    'dem08_64': "Part-time employed",
    'dem08_8': "On disability",
    'dem08_128': "Student",
    'bclinra09': "Antibiotics on enrollment",
    'bclinra10': "Hospitalization on enrollment",
    'bclinra08': 'Anthonisen Exacerbation Type',
}

df_harvard_renamed = df_harvard_med_harmo.rename(columns=harvard_rename_dict)
    
# triage_rename_dict = {
#     'Pill Exa': 'Pill to control Exa.',
#     'Gender': 'Gender, Male',
#     '2+ Exa last year': '2+ Exa. last year',
#     'Gold': 'GOLD',
#     'Kindey Disease': 'Kidney disease',
#     'FEV1 %': '% FEV1',
# }

# df_triage_renamed = df_triage_simple_harmo.rename(columns=triage_rename_dict)

# print(sorted(df_triage_renamed.columns))
# print(sorted(df_harvard_renamed.columns))

In [11]:
# only plot the variables that need to be compared
df_compare_triage = df_triage_simple_harmo.copy()
df_compare_harvard = df_harvard_renamed.copy()

# only take smokers in triage

to_be_compared_harvard = ['% FEV1 a', '% FEV1 b', 'MMRC', 'PSQI Disturbance', 'Worse cough', 'Increase in sputum', 'On disability', 'Wheezing', 'Worse dyspnea', 'On disability', 'Night Symptoms']
to_be_compared_triage = ['% FEV1 a', '% FEV1 b','MMRC a', 'MMRC b','Worse dyspnea', 'Worse cough', 'Increase in sputum', 'Lives Alone', 'Night Symptoms', 'Help Daily Activities', 'Wheezing']

# if increase in sputum is 0 or 1 then it is 0, if it is 2 or 3 then it is 1 in triage
df_compare_triage['Increase in sputum'] = 0
df_compare_triage.loc[df_compare_triage['Sputum details'] >= 2, 'Increase in sputum'] = 1
# if wheezing is 0 or 1 then it is 0, if it is 2 it is 1 in triage
df_compare_triage['Wheezing'] = 0
df_compare_triage.loc[df_compare_triage['Wheezing details'] == 2, 'Wheezing'] = 1
# if worse dyspnea is 0 or 1 then it is 0, if it is 2 it is 1 in triage
df_compare_triage['Worse dyspnea'] = 0
df_compare_triage.loc[df_compare_triage['Dyspnea details'] == 2, 'Worse dyspnea'] = 1
# if worse cough is 0 or 1 then it is 0, if it is 2 it is 1 in triage
df_compare_triage['Worse cough'] = 0
df_compare_triage.loc[df_compare_triage['Cough details'] == 2, 'Worse cough'] = 1

# create night symptoms variable in harvard that is 0 when psqi disturbance is 0, 1, 2 and 1 when psqi disturbance is 3
df_compare_harvard['Night Symptoms'] = 0
df_compare_harvard.loc[df_compare_harvard['PSQI Disturbance'] >= 2.5, 'Night Symptoms'] = 1

# # create 3x4 plot for harvard variables
# fig, axs = plt.subplots(3, 4, figsize=(15, 15))
# row = 0
# col = 0
# for i, var in enumerate(sorted(to_be_compared_harvard)):
#     if col == 4:
#         row += 1
#         col = 0
#     subset = df_compare_harvard[[var, 'Exacerbation']]
#     # plot cdf of the variable
#     sns.histplot(subset, x=var, ax=axs[row, col], hue = 'Exacerbation', kde=True, cumulative=True)
#     axs[row, col].set_title(var)
#     col += 1

# # create 3x4 plot for triage variables
# fig, axs = plt.subplots(3, 4, figsize=(15, 15))
# row = 0
# col = 0
# for i, var in enumerate(sorted(to_be_compared_triage)):
#     if col == 4:
#         row += 1
#         col = 0
    
#     subset = df_compare_triage[[var, 'Exacerbation']]
#     sns.histplot(subset, x=var, ax=axs[row, col], hue ='Exacerbation', kde=True, cumulative=True)
#     axs[row, col].set_title(var)
#     col += 1



# Merge with data after data cleaning

In [12]:
df_harvard_fe = pd.read_csv('../harvard/data/harvard_cleaned.csv', sep=";", decimal=",")
df_triage_fe = df_triage.copy()


# append harmonized features to original harvard dataset
df_harvard_fe['Night Symptoms'] = df_compare_harvard['Night Symptoms']
df_harvard_fe['Pill to control Exa.'] = df_compare_harvard['Pill to control Exa.']
df_harvard_fe['Rescue Inhaler'] = df_compare_harvard['Rescue Inhaler']
df_harvard_fe['Daily Inhaler'] = df_compare_harvard['Daily Inhaler']
df_harvard_fe['Lives Alone'] = df_compare_harvard['Lives Alone']
df_harvard_fe['Hospitalized last year'] = df_compare_harvard['Hospitalized last year']
df_harvard_fe['ICU last year'] = df_compare_harvard['ICU last year']
df_harvard_fe['2+ Exa. last year'] = df_compare_harvard['2+ Exa. last year']
df_harvard_fe['BMI'] = df_compare_harvard['BMI']
df_harvard_fe.rename(columns={'Exa. during study':'Exacerbation'}, inplace=True)

# append harmonized features to original triage dataset
df_triage_fe['Wheezing'] = df_compare_triage['Wheezing']
df_triage_fe['Worse cough'] = df_compare_triage['Worse cough']
df_triage_fe['Worse dyspnea'] = df_compare_triage['Worse dyspnea']
df_triage_fe['Increase in sputum'] = df_compare_triage['Increase in sputum']
df_triage_fe['Fever'] = df_compare_triage['Fever']

df_harvard_fe = df_harvard_fe.rename(columns=harvard_rename_dict)
# df_triage_fe = df_triage_fe.rename(columns=triage_rename_dict)

# print each variable in both datasets in alphabetical order
harvard_vars = sorted(df_harvard_fe.columns)
triage_vars = sorted(df_triage_fe.columns)
# print intersection
intersection = set(harvard_vars).intersection(triage_vars)
print(sorted(intersection))

# difference between harvard and triage
harvard_only = set(harvard_vars).difference(triage_vars)
print("harvard only: ")
print(sorted(harvard_only))
triage_only = set(triage_vars).difference(harvard_vars)
print("triage only: ")
print(sorted(triage_only))

# sort columns in both datasets
df_harvard_fe = df_harvard_fe[sorted(df_harvard_fe.columns)]
df_triage_fe = df_triage_fe[sorted(df_triage_fe.columns)]

# convert all object types to float
for col in df_harvard_fe.columns:
    if df_harvard_fe[col].dtype == 'object':
        df_harvard_fe[col] = df_harvard_fe[col].astype(float)

# export the datasets
df_harvard_fe.to_csv('./data/harvard_feature_engineered.csv', sep=";", decimal=",", index=False)
df_triage_fe.to_csv('./data/triage_feature_engineered.csv', sep=";", decimal=",", index=False)

# export the first two dimensions as tensorflow format (vectors are in rows)
df_harvard_fe['2+ Exa. last year'].to_csv('./data/harvard_feature_engineered_tf.tsv', sep=";", decimal=",", index=False, header=False)

['% FEV1 a', '% FEV1 b', '2+ Exa. last year', 'Age', 'Asthma', 'BMI', 'Congestive Heart Failure', 'Coronary Artery Disease', 'Daily Inhaler', 'Diabetes', 'Exacerbation', 'Fever', 'GOLD', 'Gender, Male', 'Height', 'Hospitalized last year', 'Hypertension', 'ICU last year', 'Increase in sputum', 'Kidney disease', 'Lives Alone', 'Night Symptoms', 'Oxygen Therapy', 'Pill to control Exa.', 'Rescue Inhaler', 'Weight', 'Wheezing', 'Worse cough', 'Worse dyspnea']
harvard only: 
['% FVC a', '% FVC b', '% PEF a', '% PEF b', '6MWT', 'Advair', 'Albuterol', 'Anthonisen Exacerbation Type', 'Antibiotics on enrollment', 'Anxiety', 'Arthritis / rheumatism', 'Assmnt', 'Beclomethasone', 'Breathing faster', 'Budesonide', 'COPD state', 'COPD type', 'Cancer', 'Chest pain', 'Combivent', 'Depression', 'ER visits last 12 months', 'Education level', 'Enrolled as Exacerbator', 'Exa. last 12 months', 'Exa. last 2 months', 'FEV1', 'FEV1 b', 'FEV1/FVC x 100', 'FEV1/FVC x 100 a', 'FVC', 'FVC b', 'Flucticasone', 'Form

In [33]:
# # plot data

# # harvard
# numeric_cols = df_harvard_fe.columns[df_harvard_fe.nunique() > 2]


# fig, axs = plt.subplots(7, 7, figsize=(100, 100))
# # plot the histogram of each variable
# row, col = 0, 0
# for var in sorted(numeric_cols):
#     # if we are at the end of the row, move to the next row
#     if col == 6:
#         row += 1
#         col = 0
#     # create 
#     subset = df_harvard_fe[[var, 'Exacerbation']]
#     # add a row called label
#     sns.histplot(subset, x=var, hue='Exacerbation', ax=axs[row, col])
#     # add a bar for the mean of each group
#     axs[row, col].axvline(subset[subset['Exacerbation'] == 0][var].mean(), color='blue', linestyle='dashed', linewidth=5, label='No Exacerbation')
#     axs[row, col].axvline(subset[subset['Exacerbation'] == 1][var].mean(), color='orange', linestyle='dashed', linewidth=5, label='Exacerbation')
#     # set title
#     axs[row, col].set_title(var)
#     # add legend
#     axs[row, col].legend(loc='upper right')
#     # move to the next column
#     col += 1

In [34]:
# # triage

# # harvard
# numeric_cols = df_triage_fe.columns[df_triage_fe.nunique() > 2]


# fig, axs = plt.subplots(7, 7, figsize=(100, 100))
# # plot the histogram of each variable
# row, col = 0, 0
# for var in sorted(numeric_cols):
#     # if we are at the end of the row, move to the next row
#     if col == 6:
#         row += 1
#         col = 0
#     # create 
#     subset = df_triage_fe[[var, 'Exacerbation']]
#     # add a row called label
#     sns.histplot(subset, x=var, hue='Exacerbation', ax=axs[row, col])
#     # add a bar for the mean of each group
#     axs[row, col].axvline(subset[subset['Exacerbation'] == 0][var].mean(), color='blue', linestyle='dashed', linewidth=5, label='No Exacerbation')
#     axs[row, col].axvline(subset[subset['Exacerbation'] == 1][var].mean(), color='orange', linestyle='dashed', linewidth=5, label='Exacerbation')
#     # set title
#     axs[row, col].set_title(var)
#     # add legend
#     axs[row, col].legend(loc='upper right')
#     # move to the next column
#     col += 1

In [35]:
# from sklearn.svm import SVC
# from sklearn.ensemble import VotingClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.naive_bayes import GaussianNB
# # try fitting triage using k-fold

# stratfied_fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# clf1 = RandomForestClassifier(random_state=42)
# clf2 = XGBClassifier(random_state=42)
# clf3 = GaussianNB()
# clf4 = SVC(probability=True, random_state=42)

# voting_clf = VotingClassifier(estimators=[('rf', clf1), ('xgb', clf2), ('svc', clf3), ()], voting='soft', )


# # define the pipeline
# pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler()),
#     ('model', clf1)
# ])

# cv_score = cross_val_score(pipeline, df_triage_fe.drop(columns=['Exacerbation']), df_triage_fe['Exacerbation'], scoring='roc_auc', cv=stratfied_fold)

# print(cv_score)


